In [82]:
import numpy as np
from qiskit import QuantumCircuit, transpile
from qiskit.circuit import Parameter
from qiskit_aer import Aer
from qiskit.quantum_info import SparsePauliOp, Statevector
from qiskit_aer.primitives import Estimator
# from qiskit_ibm_runtime import QiskitRuntimeService, EstimatorV2 as Estimator
from scipy.optimize import minimize

In [36]:
token = "29c58e575de83a0ce0febec0eeab8c518915c824691fc9c2a78fbc0d40c6e91b8cfdaebe3e7ab263dcc4a357de2b725d20798ad1030b5f1a0e9b6879d88f930c"

In [37]:
QiskitRuntimeService.save_account(
    token=token,
    channel="ibm_quantum",
    set_as_default=True,
    overwrite=True
)

In [38]:
service = QiskitRuntimeService()

In [2]:
n_qubits = 6
weights = np.random.uniform(0, 1, size=(n_qubits, n_qubits))
HC_terms = []

In [4]:
for i in range(n_qubits):
    for j in range(i):
        if weights[i, j] != 0:
            pauli_string = ['I'] * n_qubits
            pauli_string[i] = 'Z'
            pauli_string[j] = 'Z'
            HC_terms.append((''.join(pauli_string), -0.5 * weights[i, j]))

In [5]:
HC_terms

[('ZZIIII', -0.05422993719017061),
 ('ZIZIII', -0.43141594364473934),
 ('IZZIII', -0.042607711061399045),
 ('ZIIZII', -0.06324300949885275),
 ('IZIZII', -0.4483466539723786),
 ('IIZZII', -0.3188065460242888),
 ('ZIIIZI', -0.04888766325290855),
 ('IZIIZI', -0.2555462284384329),
 ('IIZIZI', -0.39776050496278015),
 ('IIIZZI', -0.4495385728264034),
 ('ZIIIIZ', -0.44768132492830487),
 ('IZIIIZ', -0.22956016833605103),
 ('IIZIIZ', -0.22376484457852663),
 ('IIIZIZ', -0.3100644471892164),
 ('IIIIZZ', -0.1719168169866901)]

In [6]:
HC = SparsePauliOp.from_list(HC_terms)

In [7]:
qc = QuantumCircuit(n_qubits)
qc.h(range(n_qubits))
initial_state = qc

In [19]:
mixer_pool = []
for i in range(n_qubits):
    pauli_string = ['I'] * n_qubits
    pauli_string[i] = 'X'
    mixer_pool.append(SparsePauliOp.from_list([(''.join(pauli_string), 1.0)]))

In [20]:
mixer_pool

[SparsePauliOp(['XIIIII'],
               coeffs=[1.+0.j]),
 SparsePauliOp(['IXIIII'],
               coeffs=[1.+0.j]),
 SparsePauliOp(['IIXIII'],
               coeffs=[1.+0.j]),
 SparsePauliOp(['IIIXII'],
               coeffs=[1.+0.j]),
 SparsePauliOp(['IIIIXI'],
               coeffs=[1.+0.j]),
 SparsePauliOp(['IIIIIX'],
               coeffs=[1.+0.j])]

In [21]:
def measure_gradient(operator, ansatz, h_c, estimator):

    commutator = (-1j * (operator @ h_c - h_c @ operator)).simplify()
    circuits = [ansatz]
    results = estimator.run([commutator], circuits).result()
    gradients = results.values
    return np.real(gradients[0])

In [22]:
params = {'gamma': [], 'beta': []}
ansatz = QuantumCircuit(n_qubits)
ansatz.compose(initial_state, inplace=True)

In [47]:
threshold = 1e-3
estimator = Estimator()

In [68]:
commutators = [(-1j * (op @ HC - HC @ op)).simplify() for op in mixer_pool]

In [70]:
commutators[0]

SparsePauliOp(['YZIIII', 'YIZIII', 'YIIZII', 'YIIIZI', 'YIIIIZ'],
              coeffs=[0.10845987+0.j, 0.86283189+0.j, 0.12648602+0.j, 0.09777533+0.j,
 0.89536265+0.j])

In [71]:
circuits = [ansatz]

In [72]:
circuits

In [60]:
operators = [commutator]

In [76]:
results = estimator.run([circuits], [commutators[0]])

TypeError: Invalid circuits, expected Sequence[QuantumCircuit].

In [78]:
# Define the cost Hamiltonian for Max-Cut
n_qubits = 6
weights = np.random.uniform(0, 1, size=(n_qubits, n_qubits))  # Random weights
H_C_terms = []
for i in range(n_qubits):
    for j in range(i):
        if weights[i, j] != 0:
            pauli_string = ["I"] * n_qubits
            pauli_string[i] = "Z"
            pauli_string[j] = "Z"
            H_C_terms.append(("".join(pauli_string), -0.5 * weights[i, j]))
H_C = SparsePauliOp.from_list(H_C_terms)

# Define the initial state |+>^n
qc = QuantumCircuit(n_qubits)
qc.h(range(n_qubits))
initial_state = qc

# Define the mixer pool
mixer_pool = []
for i in range(n_qubits):
    pauli_string = ["I"] * n_qubits
    pauli_string[i] = "X"
    mixer_pool.append(SparsePauliOp.from_list([("".join(pauli_string), 1.0)]))

def measure_gradient(operator, ansatz, h_c, estimator):
    """
    Measure the gradient of the cost function with respect to a given operator.
    """
    circuits = [ansatz]
    operators = [operator]
    results = estimator.run(operators, circuits).result()
    gradients = results.values
    return np.real(gradients[0])

def adapt_qaoa_iteration(ansatz, mixer_pool, h_c, params, estimator):
    """
    Perform one iteration of the ADAPT-QAOA algorithm.
    """
    gradients = [measure_gradient(op, ansatz, h_c, estimator) for op in mixer_pool]
    max_grad_index = np.argmax(np.abs(gradients))
    selected_operator = mixer_pool[max_grad_index]

    # Add new operator to the ansatz
    beta_param = Parameter(f"beta_{len(params['beta'])}")
    gamma_param = Parameter(f"gamma_{len(params['gamma'])}")

    layer = QuantumCircuit(n_qubits)
    layer.rz(gamma_param, max_grad_index)
    layer.rx(beta_param, max_grad_index)

    ansatz = ansatz.compose(layer)

    params['beta'].append(beta_param)
    params['gamma'].append(gamma_param)

    return ansatz, selected_operator

# Initialize parameters
params = {'gamma': [], 'beta': []}
ansatz = QuantumCircuit(n_qubits)
ansatz.compose(initial_state, inplace=True)

# Main ADAPT-QAOA loop
threshold = 1e-3
estimator = Estimator()

for iteration in range(10):
    ansatz, selected_op = adapt_qaoa_iteration(ansatz, mixer_pool, H_C, params, estimator)
    print(f"Iteration {iteration}: Added operator {selected_op}")

    # Check gradient norm
    gradients = [measure_gradient(op, ansatz, H_C, estimator) for op in mixer_pool]
    norm = np.linalg.norm(gradients)
    print(f"Gradient norm: {norm}")
    if norm < threshold:
        print("Convergence reached.")
        break

# Optimize the final ansatz
all_params = params['gamma'] + params['beta']
def cost_function(param_values):
    param_dict = dict(zip(all_params, param_values))
    ansatz_bound = ansatz.bind_parameters(param_dict)
    circuits = [ansatz_bound]
    operators = [H_C]
    result = estimator.run(operators, circuits).result()
    energy = result.values[0]
    return np.real(energy)

initial_values = np.random.uniform(0, 2 * np.pi, len(all_params))
result = minimize(cost_function, initial_values, method='COBYLA')
print(f"Optimized energy: {result.fun}")


TypeError: Invalid circuits, expected Sequence[QuantumCircuit].

In [84]:
# Define the cost Hamiltonian for Max-Cut
n_qubits = 6
weights = np.random.uniform(0, 1, size=(n_qubits, n_qubits))  # Random weights
H_C_terms = []
for i in range(n_qubits):
    for j in range(i):
        if weights[i, j] != 0:
            pauli_string = ["I"] * n_qubits
            pauli_string[i] = "Z"
            pauli_string[j] = "Z"
            H_C_terms.append(("".join(pauli_string), -0.5 * weights[i, j]))
H_C = SparsePauliOp.from_list(H_C_terms)

# Define the initial state |+>^n
qc = QuantumCircuit(n_qubits)
qc.h(range(n_qubits))  # Apply Hadamard to all qubits
initial_state = qc

# Define the mixer pool
mixer_pool = []
for i in range(n_qubits):
    pauli_string = ["I"] * n_qubits
    pauli_string[i] = "X"
    mixer_pool.append(SparsePauliOp.from_list([("".join(pauli_string), 1.0)]))

# Function to calculate the expectation value
def calculate_expectation(circuit, hamiltonian, param_values):
    """
    Calculate the expectation value of the Hamiltonian for a given circuit with bound parameters.
    """
    bound_circuit = circuit.bind_parameters(param_values)
    state = Statevector.from_instruction(bound_circuit)
    expectation = np.real(state.expectation_value(hamiltonian))
    return expectation

# Function to calculate the gradient
def measure_gradient(operator, ansatz, hamiltonian, param_values):
    """
    Measure the gradient of the cost function with respect to a given operator using finite differences.
    """
    delta = 1e-3
    param_values_plus = param_values.copy()
    param_values_minus = param_values.copy()

    # Perturb a single parameter (assume last added parameter for simplicity)
    param_key = list(param_values.keys())[-1]
    param_values_plus[param_key] += delta
    param_values_minus[param_key] -= delta

    expectation_plus = calculate_expectation(ansatz, hamiltonian, param_values_plus)
    expectation_minus = calculate_expectation(ansatz, hamiltonian, param_values_minus)

    gradient = (expectation_plus - expectation_minus) / (2 * delta)
    return gradient

# Function to perform one iteration of ADAPT-QAOA
def adapt_qaoa_iteration(ansatz, mixer_pool, params, hamiltonian, param_values):
    """
    Perform one iteration of the ADAPT-QAOA algorithm.
    """
    gradients = [measure_gradient(op, ansatz, hamiltonian, param_values) for op in mixer_pool]
    max_grad_index = np.argmax(np.abs(gradients))
    selected_operator = mixer_pool[max_grad_index]

    # Add new operator to the ansatz
    beta_param = Parameter(f"beta_{len(params['beta'])}")
    gamma_param = Parameter(f"gamma_{len(params['gamma'])}")

    layer = QuantumCircuit(n_qubits)
    layer.rz(gamma_param, max_grad_index)
    layer.rx(beta_param, max_grad_index)

    ansatz = ansatz.compose(layer)

    params['beta'].append(beta_param)
    params['gamma'].append(gamma_param)

    param_values[beta_param] = 0.0  # Initialize parameter
    param_values[gamma_param] = 0.0  # Initialize parameter

    return ansatz, selected_operator

# Initialize parameters
params = {'gamma': [], 'beta': []}
param_values = {}
ansatz = QuantumCircuit(n_qubits)
ansatz.compose(initial_state, inplace=True)

# Main ADAPT-QAOA loop
threshold = 1e-3
for iteration in range(10):
    ansatz, selected_op = adapt_qaoa_iteration(ansatz, mixer_pool, params, H_C, param_values)
    print(f"Iteration {iteration}: Added operator {selected_op}")

    # Check gradient norm
    gradients = [measure_gradient(op, ansatz, H_C, param_values) for op in mixer_pool]
    norm = np.linalg.norm(gradients)
    print(f"Gradient norm: {norm}")
    if norm < threshold:
        print("Convergence reached.")
        break

# Optimize the final ansatz
all_params = params['gamma'] + params['beta']
def cost_function(param_values_array):
    param_dict = dict(zip(all_params, param_values_array))
    energy = calculate_expectation(ansatz, H_C, param_dict)
    return energy

initial_values = np.random.uniform(0, 2 * np.pi, len(all_params))
result = minimize(cost_function, initial_values, method='COBYLA')
print(f"Optimized energy: {result.fun}")


IndexError: list index out of range

In [1]:
from qiskit import Aer, QuantumCircuit, execute
from qiskit.circuit import Parameter
from qiskit.opflow import I, X, Z, StateFn, PauliSumOp
from qiskit.algorithms.optimizers import COBYLA
import numpy as np

# Define problem parameters
n_qubits = 3  # Number of qubits (variables)
penalty_coefficient = 10.0  # Penalty term coefficient

# Define the cost Hamiltonian (objective function)
def cost_hamiltonian():
    # Example cost function: H_C = Z_0 + Z_1 + Z_2
    terms = [Z ^ I ^ I, I ^ Z ^ I, I ^ I ^ Z]
    return sum(terms)

# Define the penalty Hamiltonian (for constraints)
def penalty_hamiltonian():
    # Constraint: Sum of qubits <= 1
    terms = []
    for i in range(n_qubits):
        terms.append(Z ^ I ^ I if i == 0 else I ^ Z ^ I if i == 1 else I ^ I ^ Z)
    penalty = penalty_coefficient * sum(terms)
    return penalty

# Define the mixer Hamiltonian with penalty terms
def mixer_hamiltonian():
    # Standard X-mixer + penalty term
    standard_mixer = sum([X ^ I ^ I if i == 0 else I ^ X ^ I if i == 1 else I ^ I ^ X for i in range(n_qubits)])
    return standard_mixer + penalty_hamiltonian()

# Define the QAOA ansatz
def qaoa_ansatz(gamma, beta):
    qc = QuantumCircuit(n_qubits)

    # Start in superposition
    for q in range(n_qubits):
        qc.h(q)

    # Apply cost Hamiltonian
    for term in cost_hamiltonian().primitive:  # Pauli terms
        qubits = [i for i, pauli in enumerate(term.primitive) if pauli != "I"]
        if len(qubits) == 1:
            qc.rz(-gamma, qubits[0])

    # Apply mixer Hamiltonian
    for term in mixer_hamiltonian().primitive:
        qubits = [i for i, pauli in enumerate(term.primitive) if pauli != "I"]
        if len(qubits) == 1:
            qc.rx(-beta, qubits[0])

    return qc

# Example implementation
if __name__ == "__main__":
    # Define parameters
    gamma = Parameter("gamma")
    beta = Parameter("beta")

    # Create QAOA circuit
    qaoa_circuit = qaoa_ansatz(gamma, beta)

    # Backend and simulation
    simulator = Aer.get_backend("statevector_simulator")

    # Define a test set of parameters
    gamma_val = np.pi / 4
    beta_val = np.pi / 8

    # Bind parameters and simulate
    bound_circuit = qaoa_circuit.bind_parameters({gamma: gamma_val, beta: beta_val})
    result = execute(bound_circuit, backend=simulator).result()
    statevector = result.get_statevector()

    print("Statevector:", statevector)


/tmp/ipykernel_25148/2281040211.py:3: DeprecationWarning: The ``qiskit.opflow`` module is deprecated as of qiskit-terra 0.24.0. It will be removed in Qiskit 1.0. For code migration guidelines, visit https://qisk.it/opflow_migration.
  from qiskit.opflow import I, X, Z, StateFn, PauliSumOp
/tmp/ipykernel_25148/2281040211.py:4: DeprecationWarning: ``qiskit.algorithms`` has been migrated to an independent package: https://github.com/qiskit-community/qiskit-algorithms. The ``qiskit.algorithms`` import path is deprecated as of qiskit-terra 0.25.0 and will be removed in Qiskit 1.0. Please run ``pip install qiskit_algorithms`` and use ``import qiskit_algorithms`` instead.
  from qiskit.algorithms.optimizers import COBYLA


AttributeError: 'SparsePauliOp' object has no attribute 'primitive'

In [2]:
from qiskit import Aer, QuantumCircuit
from qiskit.circuit import Parameter
from qiskit.quantum_info import Pauli
from qiskit.primitives import Estimator
import numpy as np

# Define problem parameters
n_qubits = 3  # Number of qubits (variables)
penalty_coefficient = 10.0  # Penalty term coefficient

# Define the cost Hamiltonian (objective function)
def cost_hamiltonian():
    # Example cost function: H_C = Z_0 + Z_1 + Z_2
    terms = [Pauli("ZII"), Pauli("IZI"), Pauli("IIZ")]
    coefficients = [1.0, 1.0, 1.0]
    return terms, coefficients

# Define the penalty Hamiltonian (for constraints)
def penalty_hamiltonian():
    # Constraint: Sum of qubits <= 1
    terms = [Pauli("ZII"), Pauli("IZI"), Pauli("IIZ")]
    coefficients = [penalty_coefficient] * len(terms)
    return terms, coefficients

# Define the mixer Hamiltonian with penalty terms
def mixer_hamiltonian():
    # Standard X-mixer + penalty term
    terms = [Pauli("XII"), Pauli("IXI"), Pauli("IIX")]
    coefficients = [1.0] * len(terms)

    penalty_terms, penalty_coefficients = penalty_hamiltonian()

    terms += penalty_terms
    coefficients += penalty_coefficients

    return terms, coefficients

# Define the QAOA ansatz
def qaoa_ansatz(gamma, beta):
    qc = QuantumCircuit(n_qubits)

    # Start in superposition
    for q in range(n_qubits):
        qc.h(q)

    # Apply cost Hamiltonian
    terms, coefficients = cost_hamiltonian()
    for term, coeff in zip(terms, coefficients):
        indices = [i for i, pauli in enumerate(term.to_label()) if pauli != "I"]
        for idx in indices:
            qc.rz(-gamma * coeff, idx)

    # Apply mixer Hamiltonian
    terms, coefficients = mixer_hamiltonian()
    for term, coeff in zip(terms, coefficients):
        indices = [i for i, pauli in enumerate(term.to_label()) if pauli != "I"]
        for idx in indices:
            qc.rx(-beta * coeff, idx)

    return qc

# Example implementation
if __name__ == "__main__":
    # Define parameters
    gamma = Parameter("gamma")
    beta = Parameter("beta")

    # Create QAOA circuit
    qaoa_circuit = qaoa_ansatz(gamma, beta)

    # Backend and simulation
    simulator = Aer.get_backend("statevector_simulator")

    # Define a test set of parameters
    gamma_val = np.pi / 4
    beta_val = np.pi / 8

    # Bind parameters and simulate
    bound_circuit = qaoa_circuit.bind_parameters({gamma: gamma_val, beta: beta_val})
    result = simulator.run(bound_circuit).result()
    statevector = result.get_statevector()

    print("Statevector:", statevector)


/tmp/ipykernel_25148/3722845864.py:72: DeprecationWarning: The 'qiskit.Aer' entry point is deprecated and will be removed in Qiskit 1.0. You should use 'qiskit_aer.Aer' directly instead.
  simulator = Aer.get_backend("statevector_simulator")


Statevector: Statevector([0.06124377-0.03820001j, 0.15471552-0.03010812j,
             0.15471552-0.03010812j, 0.33902889+0.05934515j,
             0.15471552-0.03010812j, 0.33902889+0.05934515j,
             0.33902889+0.05934515j, 0.64510816+0.38562668j],
            dims=(2, 2, 2))


/tmp/ipykernel_25148/3722845864.py:79: DeprecationWarning: The method ``qiskit.circuit.quantumcircuit.QuantumCircuit.bind_parameters()`` is deprecated as of qiskit 0.45.0. It will be removed in the Qiskit 1.0.0 release. Use assign_parameters() instead
  bound_circuit = qaoa_circuit.bind_parameters({gamma: gamma_val, beta: beta_val})


In [3]:
from qiskit import Aer, QuantumCircuit
from qiskit.circuit import Parameter
from qiskit.quantum_info import Pauli, SparsePauliOp
from qiskit.primitives import Estimator
import numpy as np

# Define problem parameters
n_qubits = 3  # Number of qubits (variables)
penalty_coefficient = 10.0  # Penalty term coefficient

# Define the cost Hamiltonian (objective function)
def cost_hamiltonian():
    # Example cost function: H_C = Z_0 + Z_1 + Z_2
    terms = [Pauli("ZII"), Pauli("IZI"), Pauli("IIZ")]
    coefficients = [1.0, 1.0, 1.0]
    return terms, coefficients

# Define the penalty Hamiltonian (for constraints)
def penalty_hamiltonian():
    # Constraint: Sum of qubits <= 1
    terms = [Pauli("ZII"), Pauli("IZI"), Pauli("IIZ")]
    coefficients = [penalty_coefficient] * len(terms)
    return terms, coefficients

# Define the mixer Hamiltonian with penalty terms
def mixer_hamiltonian():
    # Standard X-mixer + penalty term
    terms = [Pauli("XII"), Pauli("IXI"), Pauli("IIX")]
    coefficients = [1.0] * len(terms)

    penalty_terms, penalty_coefficients = penalty_hamiltonian()

    terms += penalty_terms
    coefficients += penalty_coefficients

    return terms, coefficients

# Define the QAOA ansatz
def qaoa_ansatz(gamma, beta):
    qc = QuantumCircuit(n_qubits)

    # Start in superposition
    for q in range(n_qubits):
        qc.h(q)

    # Apply cost Hamiltonian
    terms, coefficients = cost_hamiltonian()
    for term, coeff in zip(terms, coefficients):
        indices = [i for i, pauli in enumerate(term.to_label()) if pauli != "I"]
        for idx in indices:
            qc.rz(-gamma * coeff, idx)

    # Apply mixer Hamiltonian
    terms, coefficients = mixer_hamiltonian()
    for term, coeff in zip(terms, coefficients):
        indices = [i for i, pauli in enumerate(term.to_label()) if pauli != "I"]
        for idx in indices:
            qc.rx(-beta * coeff, idx)

    return qc

# Compute the MaxCut value
def compute_maxcut(statevector, cost_terms, cost_coefficients):
    maxcut_value = 0
    probabilities = np.abs(statevector) ** 2

    for i, prob in enumerate(probabilities):
        bitstring = format(i, f"0{n_qubits}b")  # Convert index to bitstring
        energy = 0

        for term, coeff in zip(cost_terms, cost_coefficients):
            term_value = 1
            for idx, pauli in enumerate(term.to_label()):
                if pauli == "Z":
                    term_value *= 1 if bitstring[idx] == "0" else -1
            energy += coeff * term_value

        maxcut_value += prob * energy

    return maxcut_value

# Example implementation
if __name__ == "__main__":
    # Define parameters
    gamma = Parameter("gamma")
    beta = Parameter("beta")

    # Create QAOA circuit
    qaoa_circuit = qaoa_ansatz(gamma, beta)

    # Backend and simulation
    simulator = Aer.get_backend("statevector_simulator")

    # Define a test set of parameters
    gamma_val = np.pi / 4
    beta_val = np.pi / 8

    # Bind parameters and simulate
    bound_circuit = qaoa_circuit.bind_parameters({gamma: gamma_val, beta: beta_val})
    result = simulator.run(bound_circuit).result()
    statevector = result.get_statevector()

    # Compute MaxCut value
    cost_terms, cost_coefficients = cost_hamiltonian()
    maxcut_value = compute_maxcut(statevector, cost_terms, cost_coefficients)

    print("Statevector:", statevector)
    print("MaxCut Value:", maxcut_value)


Statevector: Statevector([0.06124377-0.03820001j, 0.15471552-0.03010812j,
             0.15471552-0.03010812j, 0.33902889+0.05934515j,
             0.15471552-0.03010812j, 0.33902889+0.05934515j,
             0.33902889+0.05934515j, 0.64510816+0.38562668j],
            dims=(2, 2, 2))
MaxCut Value: -1.959844447314564


/tmp/ipykernel_25148/1211750165.py:99: DeprecationWarning: The method ``qiskit.circuit.quantumcircuit.QuantumCircuit.bind_parameters()`` is deprecated as of qiskit 0.45.0. It will be removed in the Qiskit 1.0.0 release. Use assign_parameters() instead
  bound_circuit = qaoa_circuit.bind_parameters({gamma: gamma_val, beta: beta_val})


In [4]:
from qiskit import Aer, QuantumCircuit
from qiskit.circuit import Parameter
from qiskit.quantum_info import Pauli, SparsePauliOp
from qiskit.primitives import Estimator
import numpy as np
from itertools import product

# Define problem parameters
n_qubits = 3  # Number of qubits (variables)
penalty_coefficient = 10.0  # Penalty term coefficient

# Define the cost Hamiltonian (objective function)
def cost_hamiltonian():
    # Example cost function: H_C = Z_0 + Z_1 + Z_2
    terms = [Pauli("ZII"), Pauli("IZI"), Pauli("IIZ")]
    coefficients = [1.0, 1.0, 1.0]
    return terms, coefficients

# Define the penalty Hamiltonian (for constraints)
def penalty_hamiltonian():
    # Constraint: Sum of qubits <= 1
    terms = [Pauli("ZII"), Pauli("IZI"), Pauli("IIZ")]
    coefficients = [penalty_coefficient] * len(terms)
    return terms, coefficients

# Define the mixer Hamiltonian with penalty terms
def mixer_hamiltonian():
    # Standard X-mixer + penalty term
    terms = [Pauli("XII"), Pauli("IXI"), Pauli("IIX")]
    coefficients = [1.0] * len(terms)

    penalty_terms, penalty_coefficients = penalty_hamiltonian()

    terms += penalty_terms
    coefficients += penalty_coefficients

    return terms, coefficients

# Define the QAOA ansatz
def qaoa_ansatz(gamma, beta):
    qc = QuantumCircuit(n_qubits)

    # Start in superposition
    for q in range(n_qubits):
        qc.h(q)

    # Apply cost Hamiltonian
    terms, coefficients = cost_hamiltonian()
    for term, coeff in zip(terms, coefficients):
        indices = [i for i, pauli in enumerate(term.to_label()) if pauli != "I"]
        for idx in indices:
            qc.rz(-gamma * coeff, idx)

    # Apply mixer Hamiltonian
    terms, coefficients = mixer_hamiltonian()
    for term, coeff in zip(terms, coefficients):
        indices = [i for i, pauli in enumerate(term.to_label()) if pauli != "I"]
        for idx in indices:
            qc.rx(-beta * coeff, idx)

    return qc

# Compute the MaxCut value
def compute_maxcut(statevector, cost_terms, cost_coefficients):
    maxcut_value = 0
    probabilities = np.abs(statevector) ** 2

    for i, prob in enumerate(probabilities):
        bitstring = format(i, f"0{n_qubits}b")  # Convert index to bitstring
        energy = 0

        for term, coeff in zip(cost_terms, cost_coefficients):
            term_value = 1
            for idx, pauli in enumerate(term.to_label()):
                if pauli == "Z":
                    term_value *= 1 if bitstring[idx] == "0" else -1
            energy += coeff * term_value

        maxcut_value += prob * energy

    return maxcut_value

# Compute the classical MaxCut solution
def classical_maxcut(cost_terms, cost_coefficients):
    max_energy = float("-inf")
    best_bitstring = None

    for bitstring in product([0, 1], repeat=n_qubits):
        energy = 0
        for term, coeff in zip(cost_terms, cost_coefficients):
            term_value = 1
            for idx, pauli in enumerate(term.to_label()):
                if pauli == "Z":
                    term_value *= 1 if bitstring[idx] == 0 else -1
            energy += coeff * term_value

        if energy > max_energy:
            max_energy = energy
            best_bitstring = bitstring

    return max_energy, best_bitstring

# Example implementation
if __name__ == "__main__":
    # Define parameters
    gamma = Parameter("gamma")
    beta = Parameter("beta")

    # Create QAOA circuit
    qaoa_circuit = qaoa_ansatz(gamma, beta)

    # Backend and simulation
    simulator = Aer.get_backend("statevector_simulator")

    # Define a test set of parameters
    gamma_val = np.pi / 4
    beta_val = np.pi / 8

    # Bind parameters and simulate
    bound_circuit = qaoa_circuit.bind_parameters({gamma: gamma_val, beta: beta_val})
    result = simulator.run(bound_circuit).result()
    statevector = result.get_statevector()

    # Compute MaxCut value
    cost_terms, cost_coefficients = cost_hamiltonian()
    maxcut_value = compute_maxcut(statevector, cost_terms, cost_coefficients)

    # Compute classical MaxCut solution
    classical_value, classical_solution = classical_maxcut(cost_terms, cost_coefficients)

    print("Statevector:", statevector)
    print("Quantum MaxCut Value:", maxcut_value)
    print("Classical MaxCut Value:", classical_value)
    print("Classical Solution:", classical_solution)


Statevector: Statevector([0.06124377-0.03820001j, 0.15471552-0.03010812j,
             0.15471552-0.03010812j, 0.33902889+0.05934515j,
             0.15471552-0.03010812j, 0.33902889+0.05934515j,
             0.33902889+0.05934515j, 0.64510816+0.38562668j],
            dims=(2, 2, 2))
Quantum MaxCut Value: -1.959844447314564
Classical MaxCut Value: 3.0
Classical Solution: (0, 0, 0)


/tmp/ipykernel_25148/1343962759.py:120: DeprecationWarning: The method ``qiskit.circuit.quantumcircuit.QuantumCircuit.bind_parameters()`` is deprecated as of qiskit 0.45.0. It will be removed in the Qiskit 1.0.0 release. Use assign_parameters() instead
  bound_circuit = qaoa_circuit.bind_parameters({gamma: gamma_val, beta: beta_val})


In [5]:
from qiskit import Aer, QuantumCircuit
from qiskit.circuit import Parameter
from qiskit.quantum_info import Pauli, SparsePauliOp
from qiskit.primitives import Estimator
import numpy as np
from itertools import product

# Define problem parameters
n_qubits = 5  # Number of qubits (variables)
penalty_coefficient = 10.0  # Penalty term coefficient

# Define the cost Hamiltonian (objective function)
def cost_hamiltonian():
    # Example cost function: MaxCut for a larger graph
    # Graph edges: (0-1), (1-2), (2-3), (3-4), (4-0)
    terms = [Pauli("ZZIII"), Pauli("IZZII"), Pauli("IIZZI"), Pauli("IIIZZ"), Pauli("ZIIIZ")]
    coefficients = [1.0, 1.0, 1.0, 1.0, 1.0]  # Each edge contributes equally
    return terms, coefficients

# Define the penalty Hamiltonian (for constraints)
def penalty_hamiltonian():
    # Constraint: At most 2 qubits can be in the |1> state
    terms = []
    coefficients = []

    for i in range(n_qubits):
        terms.append(Pauli("Z" + "I" * (n_qubits - i - 1)))
        coefficients.append(penalty_coefficient)

    return terms, coefficients

# Define the mixer Hamiltonian with penalty terms
def mixer_hamiltonian():
    # Standard X-mixer + penalty term
    terms = [Pauli("X" + "I" * (n_qubits - i - 1)) for i in range(n_qubits)]
    coefficients = [1.0] * len(terms)

    penalty_terms, penalty_coefficients = penalty_hamiltonian()

    terms += penalty_terms
    coefficients += penalty_coefficients

    return terms, coefficients

# Define the QAOA ansatz
def qaoa_ansatz(gamma, beta):
    qc = QuantumCircuit(n_qubits)

    # Start in superposition
    for q in range(n_qubits):
        qc.h(q)

    # Apply cost Hamiltonian
    terms, coefficients = cost_hamiltonian()
    for term, coeff in zip(terms, coefficients):
        indices = [i for i, pauli in enumerate(term.to_label()) if pauli != "I"]
        for idx in indices:
            qc.rz(-gamma * coeff, idx)

    # Apply mixer Hamiltonian
    terms, coefficients = mixer_hamiltonian()
    for term, coeff in zip(terms, coefficients):
        indices = [i for i, pauli in enumerate(term.to_label()) if pauli != "I"]
        for idx in indices:
            qc.rx(-beta * coeff, idx)

    return qc

# Compute the MaxCut value
def compute_maxcut(statevector, cost_terms, cost_coefficients):
    maxcut_value = 0
    probabilities = np.abs(statevector) ** 2

    for i, prob in enumerate(probabilities):
        bitstring = format(i, f"0{n_qubits}b")  # Convert index to bitstring
        energy = 0

        for term, coeff in zip(cost_terms, cost_coefficients):
            term_value = 1
            for idx, pauli in enumerate(term.to_label()):
                if pauli == "Z":
                    term_value *= 1 if bitstring[idx] == "0" else -1
            energy += coeff * term_value

        maxcut_value += prob * energy

    return maxcut_value

# Compute the classical MaxCut solution
def classical_maxcut(cost_terms, cost_coefficients):
    max_energy = float("-inf")
    best_bitstring = None

    for bitstring in product([0, 1], repeat=n_qubits):
        energy = 0
        for term, coeff in zip(cost_terms, cost_coefficients):
            term_value = 1
            for idx, pauli in enumerate(term.to_label()):
                if pauli == "Z":
                    term_value *= 1 if bitstring[idx] == 0 else -1
            energy += coeff * term_value

        if energy > max_energy:
            max_energy = energy
            best_bitstring = bitstring

    return max_energy, best_bitstring

# Example implementation
if __name__ == "__main__":
    # Define parameters
    gamma = Parameter("gamma")
    beta = Parameter("beta")

    # Create QAOA circuit
    qaoa_circuit = qaoa_ansatz(gamma, beta)

    # Backend and simulation
    simulator = Aer.get_backend("statevector_simulator")

    # Define a test set of parameters
    gamma_val = np.pi / 4
    beta_val = np.pi / 8

    # Bind parameters and simulate
    bound_circuit = qaoa_circuit.bind_parameters({gamma: gamma_val, beta: beta_val})
    result = simulator.run(bound_circuit).result()
    statevector = result.get_statevector()

    # Compute MaxCut value
    cost_terms, cost_coefficients = cost_hamiltonian()
    maxcut_value = compute_maxcut(statevector, cost_terms, cost_coefficients)

    # Compute classical MaxCut solution
    classical_value, classical_solution = classical_maxcut(cost_terms, cost_coefficients)

    print("Statevector:", statevector)
    print("Quantum MaxCut Value:", maxcut_value)
    print("Classical MaxCut Value:", classical_value)
    print("Classical Solution:", classical_solution)


Statevector: Statevector([ 0.14698445+0.14698445j, -0.09821187+0.09821187j,
              0.14698445-0.14698445j,  0.09821187+0.09821187j,
              0.14698445-0.14698445j,  0.09821187+0.09821187j,
             -0.14698445-0.14698445j,  0.09821187-0.09821187j,
              0.14698445-0.14698445j,  0.09821187+0.09821187j,
             -0.14698445-0.14698445j,  0.09821187-0.09821187j,
             -0.14698445-0.14698445j,  0.09821187-0.09821187j,
             -0.14698445+0.14698445j, -0.09821187-0.09821187j,
              0.14698445-0.14698445j,  0.09821187+0.09821187j,
             -0.14698445-0.14698445j,  0.09821187-0.09821187j,
             -0.14698445-0.14698445j,  0.09821187-0.09821187j,
             -0.14698445+0.14698445j, -0.09821187-0.09821187j,
             -0.14698445-0.14698445j,  0.09821187-0.09821187j,
             -0.14698445+0.14698445j, -0.09821187-0.09821187j,
             -0.14698445+0.14698445j, -0.09821187-0.09821187j,
              0.14698445+0.14698445j, -0.0

/tmp/ipykernel_25148/850117461.py:126: DeprecationWarning: The method ``qiskit.circuit.quantumcircuit.QuantumCircuit.bind_parameters()`` is deprecated as of qiskit 0.45.0. It will be removed in the Qiskit 1.0.0 release. Use assign_parameters() instead
  bound_circuit = qaoa_circuit.bind_parameters({gamma: gamma_val, beta: beta_val})
